# Install needed libraries

In [1]:
%%capture
!pip install transformers sacremoses newspaper3k streamlit

# Install localtunnel

In [2]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.762s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



# Create a streamlit app

In [3]:
%%writefile app.py
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
import streamlit as st
import newspaper
import nltk

nltk.download('punkt')

# Load the models and tokenizers
summarizer_model_name = "facebook/bart-large-cnn"
translator_model_name = "facebook/wmt19-en-ru"
tokenizer_name = "facebook/bart-large-cnn"

summarizer_tokenizer = AutoTokenizer.from_pretrained(summarizer_model_name)
summarizer_model = AutoModelForSeq2SeqLM.from_pretrained(summarizer_model_name)

translator_tokenizer = AutoTokenizer.from_pretrained(translator_model_name)
translator_model = AutoModelForSeq2SeqLM.from_pretrained(translator_model_name)


def summarize_text(text, tokenizer, model, max_length=512, min_length=30):
    inputs = tokenizer(text, max_length=max_length, truncation=True, return_tensors='pt')
    summary_ids = model.generate(inputs.input_ids, num_beams=4, max_length=max_length, min_length=min_length, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


def translate_text(text, tokenizer, model, max_length=512):
    inputs = tokenizer(text, max_length=max_length, truncation=True, return_tensors='pt')
    translation_ids = model.generate(inputs.input_ids, num_beams=4, max_length=max_length, early_stopping=True)
    translation = tokenizer.decode(translation_ids[0], skip_special_tokens=True)
    return translation


# Define the streamlit app
st.title('Article Summarizer and Translation')

url = st.text_input('', placeholder='Paste the URL of the article and press Enter')

if url:
    try:
        article = newspaper.Article(url)
        article.download()
        article.parse()

        img = article.top_image
        st.image(img)

        title = article.title
        st.subheader(title)

        article.nlp()

        tab1, tab2, tab3 = st.columns(3)

        with tab1:
            st.subheader('Full Text')
            txt = article.text.replace('Advertisement', '')
            st.write(txt)

        with tab2:
            st.subheader('Summary')
            summary = summarize_text(txt, summarizer_tokenizer, summarizer_model)
            st.write(summary)

        with tab3:
            st.subheader('Russian Translation')
            translation = translate_text(summary, translator_tokenizer, translator_model)
            st.write(translation)

    except:
        st.error('Sorry, something went wrong')


Writing app.py


# Run streamlit in background and expose the port 8501
Then just click in the url showed.
A log.txtfile will be created.

In [4]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.86.166.111
npx: installed 22 in 2.114s
your url is: https://quiet-items-behave.loca.lt
